In [132]:
import pickle
import numpy as np
import pandas as pd

In [133]:
# Load the Housing dataset
housing = pd.DataFrame(pd.read_csv("Housing.csv"))

In [134]:
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [135]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [136]:
housing.isnull().sum()

price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64

In [137]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Defining the map function
def binary_map(x):
    return x.map({'yes': 1, "no": 0})

# Applying the function to the housing list
housing[varlist] = housing[varlist].apply(binary_map)

In [138]:
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [139]:
# Get the dummy variables for the feature 'furnishingstatus' and store it in a new variable - 'status'
status = pd.get_dummies(housing['furnishingstatus'], drop_first = True)

housing = housing.drop(['furnishingstatus'], axis = 1)  

# Add the results to the original housing dataframe
housing = pd.concat([housing, status], axis = 1)

In [140]:
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,semi-furnished,unfurnished
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,False,False
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,False,False
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,True,False
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,False,False
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,False,False


In [141]:
from sklearn.model_selection import train_test_split

# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
df_train, df_test = train_test_split(housing, train_size = 0.7, test_size = 0.3, random_state = 40)

In [142]:
from sklearn.preprocessing import MinMaxScaler

In [143]:
scaler = MinMaxScaler()

# Apply scaler() to all the columns except the 'yes-no', 'dummy' and 'target(price)'
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])

with open("app/scaler_feature_housing.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [144]:
# Apply scaler() to target(price)
num_vars = ['price']

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])
with open("app/scaler_target_housing.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [145]:
df_train.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,semi-furnished,unfurnished
6,0.800000,0.466946,0.6,0.666667,1.000000,1,0,0,0,1,0.666667,1,True,False
381,0.176667,0.146555,0.2,0.000000,0.000000,1,0,0,0,0,0.000000,1,True,False
2,1.000000,0.563484,0.4,0.333333,0.333333,1,0,1,0,0,0.666667,1,True,False
166,0.340000,0.412382,0.4,0.000000,0.000000,1,0,1,0,1,0.666667,1,False,True
503,0.086667,0.146555,0.4,0.000000,0.000000,1,0,0,0,0,0.000000,0,True,False


In [146]:
y_train = df_train['price']
X_train = df_train.drop(['price'], axis = 1)

In [147]:
y_train.head()

6      0.800000
381    0.176667
2      1.000000
166    0.340000
503    0.086667
Name: price, dtype: float64

In [148]:
X_train.head()

# After dummies
final_features = X_train.columns.tolist()

# Save it
with open("app/feature_columns.pkl", "wb") as f:
    pickle.dump(final_features, f)

In [ ]:
# Importing LinearRegression
from sklearn.linear_model import LinearRegression

In [150]:
# Running RFE with the output number of the variable equal to 10
model = LinearRegression()
model.fit(X_train, y_train)

with open("app/model_housing.pkl", "wb") as f:
    pickle.dump(model, f)
